# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 56 new papers today
          24 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/23 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2208.10503


extracting tarball to tmp_2208.10503...

 done.


/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:366: LatexWarning: Error parsing the document directly. Trying to recover.
  warnings.warn(LatexWarning(f"Error parsing the document directly. Trying to recover."))


...

extracting tarball to tmp_2208.11063...

 done.
Retrieving document from  https://arxiv.org/e-print/2208.11068


extracting tarball to tmp_2208.11068...

 done.


/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3398: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3398: LatexWarning: Found documentclass in tmp_2208.11068/bfields.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'tables/correlations-table' from 'tmp_2208.11068/tables/correlations-table.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'tables/radial' from 'tmp_2208.11068/tables/radial.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.13/x

### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.10528-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.10528) | **Code Comparison in Galaxy Scale Simulations with Resolved Supernova  Feedback:\\ Lagrangian vs. Eulerian Methods**  |
|| Chia-Yu Hu, et al. -- incl., <mark>Matthew C. Smith</mark> |
|*Appeared on*| *2022-08-24*|
|*Comments*| *submitted; comments welcome*|
|**Abstract**| We present a suite of high-resolution simulations of an isolated dwarf galaxy using four different hydrodynamical codes: {\sc Gizmo}, {\sc Arepo}, {\sc Gadget}, and {\sc Ramses}. All codes adopt the same physical model which includes radiative cooling, photoelectric heating, star formation, and supernova (SN) feedback. Individual SN explosions are directly resolved without resorting to sub-grid models, eliminating one of the major uncertainties in cosmological simulations. We find reasonable agreement on the time-averaged star formation rates as well as the joint density-temperature distributions between all codes. However, the Lagrangian codes show significantly burstier star formation, larger supernova-driven bubbles, and stronger galactic outflows compared to the Eulerian code. This is caused by the behavior in the dense, collapsing gas clouds when the Jeans length becomes unresolved: gas in Lagrangian codes collapses to much higher densities than in Eulerian codes, as the latter is stabilized by the minimal cell size. Therefore, more of the gas cloud is converted to stars and SNe are much more clustered in the Lagrangian models, amplifying their dynamical impact. The differences between Lagrangian and Eulerian codes can be reduced by adopting a higher star formation efficiency in Eulerian codes, which significantly enhances SN clustering in the latter. Adopting a zero SN delay time reduces burstiness in all codes, resulting in vanishing outflows as SN clustering is suppressed. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.10726-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.10726) | **Truncated accretion discs in black hole X-ray binaries: dynamics and  variability signatures**  |
|| Indu K. Dihingia, Bhargav Vaidya, <mark>Christian Fendt</mark> |
|*Appeared on*| *2022-08-24*|
|*Comments*| *19 pages, 25 figures*|
|**Abstract**| Variable features in black hole X-ray Binaries (BH-XRBs) are observed in different energy ranges and time scales. The physical origin of different spectral states in BH-XRBs and their relations with the underlying accretion disc are still elusive. To investigate the intermediate state of BH-XRBs during outburst, we simulate a truncated accretion disc around a Kerr black hole using a general relativistic magneto-hydrodynamical (GRMHD) framework under axisymmetry with adaptively refined mesh. Additionally, we have also carried out radiative transfer calculations for understanding the implications of disc dynamics on emission. Dynamically, the inner edge of the truncated accretion disc oscillates in a quasi-periodic fashion (QPO). The QPO frequency of oscillations $(\nu_{\rm QPO, max})$ increases as the magnetic field strength and magnetic resistivity increase. However, as the truncation radius increases, $\nu_{\rm QPO, max}$ decreases. In our simulation models, frequency varies between $7\times(10M_{\odot}/M_{\rm BH})$ Hz $\lesssim\nu_{\rm QPO, max}\lesssim20 \times (10M_{\odot}/M_{\rm BH})$ Hz, which is in the range of low-frequency QPOs. We further find evidence of transient shocks in the highly accreting stage during oscillation. Such a transient shock acts as an extended hot post-shock corona around the black hole that has an impact on its radiative properties. The radiative transfer calculations show signatures of these oscillations in the form of modulation in the edge-brightened structure of the accretion disc. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.10854-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.10854) | **TOI-4562 b: A highly eccentric temperate Jupiter analog orbiting a young  field star**  |
|| <mark>Alexis Heitzmann</mark>, et al. -- incl., <mark>Melissa Hobson</mark>, <mark>Coel Hellier</mark>, <mark>Carl Ziegler</mark> |
|*Appeared on*| *2022-08-24*|
|*Comments*| *23 pages, 12 figures, 4 tables. Submitted to the Astrophysical Journal (23/08/2022)*|
|**Abstract**| We report the discovery of TOI-4562 b (TIC-349576261), a Jovian planet orbiting a young F7V-type star, younger than the Praesepe/Hyades clusters ($\sim$ 300-400 Myr). This planet stands out because of its unusually long orbital period for transiting planets with known masses ($P_{\mathrm{orb}}$ = $225.11757^{+0.00027}_{-0.00025}$ days), and because it has a substantial eccentricity ($e$ = $0.81^{+0.05}_{-0.05}$). The location of TOI-4562 near the southern continuous viewing zone of TESS allowed observations throughout 25 sectors, enabling an unambiguous period measurement from TESS alone. Alongside the four available TESS transits, we performed follow-up photometry using the South African Astronomical Observatory node of the Las Cumbres Observatory, and spectroscopy with the CHIRON spectrograph on the 1.5 m SMARTS telescope. We measure a radius of $1.072_{-0.043}^{+0.044}$ R$_{\mathrm{Jup}}$, and a mass of $3.29^{+1.88}_{-0.82}$ M$_{\mathrm{Jup}}$,for TOI-4562 b. The radius of the planet is consistent with contraction models describing the early evolution of the size of giant planets. We detect tentative transit timing variations at the $\sim$ 20 min level from five transit events, favouring the presence of a companion that could explain the dynamical history of this system if confirmed by future follow-up observations. With its current orbital configuration, tidal timescales are too long for TOI-4562 b to become a hot-Jupiter via high eccentricity migration, though it is not excluded that interactions with the possible companion could modify TOI-4562 b's eccentricity and trigger circularization. The characterisation of more such young systems is essential to set constraints on models describing giant planet evolution. |

## Failed papers

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.11063-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.11063) | **Towards 21-cm Intensity Mapping at $z=2.28$ with uGMRT using the Tapered  Gridded Estimator I: Foreground Avoidance**  |
|| Srijita Pal, et al. -- incl., <mark>Abhik Ghosh</mark> |
|*Appeared on*| *2022-08-24*|
|*Comments*| *13 pages, 11 figures, accepted for publication in MNRAS*|
|**Abstract**| The post-reionization $(z \le 6)$ neutral hydrogen (HI) 21-cm intensity mapping signal holds the potential to probe the large scale structures, study the expansion history and constrain various cosmological parameters. Here we apply the Tapered Gridded Estimator (TGE) to estimate $P(k_{\perp},k_{\parallel})$ the power spectrum of the $z = 2.28$ $(432.8\, {\rm MHz})$ redshifted 21-cm signal using a $24.4\,{\rm MHz}$ sub-band drawn from uGMRT Band 3 observations of European Large-Area ISO Survey-North 1 (ELAIS-N1). The TGE allows us to taper the sky response which suppresses the foreground contribution from sources in the periphery of the telescope's field of view. We apply the TGE on the measured visibility data to estimate the multi-frequency angular power spectrum (MAPS) $C_{\ell}(\Delta\nu)$ from which we determine $P(k_{\perp},k_{\parallel})$ using maximum-likelihood which naturally overcomes the issue of missing frequency channels (55 \% here). The entire methodology is validated using simulations. For the data, using the foreground avoidance technique, we obtain a $2\,\sigma$ upper limit of $\Delta^2(k) \le (133.97)^2 \, {\rm mK}^{2}$ for the 21-cm brightness temperature fluctuation at $k = 0.347 \, \textrm{Mpc}^{-1}$. This corresponds to $[\Omega_{\rm HI}b_{\rm HI}] \le 0.23$, where $\Omega_{\rm HI}$ and $b_{\rm HI}$ respectively denote the cosmic \HI mass density and the \HI bias parameter. A previous work has analyzed $8 \, {\rm MHz}$ of the same data at $z=2.19$, and reported $\Delta^{2}(k) \le (61.49)^{2} \, {\rm mK}^{2}$ and $[\Omega_{\rm HI} b_{\rm HI}] \le 0.11$ at $k=1 \, {\rm Mpc}^{-1}$. The upper limits presented here are still orders of magnitude larger than the expected signal corresponding to $\Omega_{\rm HI} \sim 10^{-3}$ and $b_{\rm HI} \sim 2 $. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.11068-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.11068) | **Nearby galaxies in the LOFAR Two-metre Sky Survey II. The magnetic  field-gas relation**  |
|| <mark>V. Heesen</mark>, et al. |
|*Appeared on*| *2022-08-24*|
|*Comments*| *Accepted to Astronomy and Astrophysics. 13 pages plus appendix including atlas of magnetic fields*|
|**Abstract**| Context. Magnetic fields are key to understand galaxy evolution, regulating stellar feedback and star formation in galaxies. Aims. We probe the origin of magnetic fields in late-type galaxies, measuring magnetic field strengths, exploring whether magnetic fields are only passive constituents of the interstellar medium, or whether they are active constituents being part of the local energy equilibrium. Methods. We measure equipartition magnetic field strengths in 39 galaxies from LoTSS-DR2 using LOFAR observations at 144 MHz with 6 arcsec angular resolution which (0.1-0.7 kpc). For a subset of 9 galaxies, we obtain atomic and molecular mass surface densities using HI and CO(2-1) data, from the THINGS and HERACLES surveys, respectively. These data are at 13 arcsec angular resolution, which corresponds to 0.3-1.2 kpc at the distances of our galaxies. We measure kinetic energy densities using HI and CO velocity dispersions. Results. We found a mean magnetic field strength of 3.6-12.5 $\mu$G with a mean of $7.9 \pm 2.0$ $\mu$G across the full sample. The magnetic field strength has the tightest and steepest relation with the total gas surface density with $B\propto \Sigma_{\rm HI+H2}^{0.309\pm0.006}$. The relation with the star-formation rate surface density and molecular gas surface density has significantly flatter slopes. After accounting for the influence of cosmic-ray transport, we found an even steeper relation of $B\propto \Sigma_{\rm HI+H2}^{0.393\pm0.009}$. Conclusions. These results suggest that the magnetic field is regulated by a $B$-$\rho$ relation, which has its origin in the saturation of the small-scale dynamo. This is borne out by an agreement of kinetic and magnetic energy densities although local deviations do exist in particular in areas of high kinetic energy densities where the magnetic field is sub-dominant. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.10845-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.10845) | **Locating dust and molecules in the inner circumstellar environment of  R~Sculptoris with MATISSE**  |
|| Julien Drevon, et al. -- incl., <mark>M. Heininger</mark>, <mark>Thomas Henning</mark>, <mark>Jacob W. Isbell</mark>, <mark>Walter Jaffe</mark>, <mark>Michael Lehmitz</mark>, <mark>Roy van Boekel</mark> |
|*Appeared on*| *2022-08-24*|
|*Comments*| *19 pages, published in A&A*|
|**Abstract**| AGB stars are one of the main sources of dust production in the Galaxy. However, it is not clear what this process looks like and where the dust is condensing in the circumstellar environment. By characterizing the location of the dust and the molecules in the close environment of an AGB star, we aim to achieve a better understanding the history of the dust formation process. We observed the carbon star R Scl with the VLTI-MATISSE instrument in L- and N-bands. The high angular resolution of the VLTI observations, combined with a large uv-plane coverage allowed us to use image reconstruction methods. To constrain the dust and molecules' location, we used two different methods: MIRA image reconstruction and the 1D code RHAPSODY. We found evidence of C2H2 and HCN molecules between 1 and 3.4 Rstar which is much closer to the star than the location of the dust (between 3.8 and 17.0 Rstar). We also estimated a mass-loss rate of 1.2+-0.4x10-6 Msun per yr. In the meantime, we confirmed the previously published characteristics of a thin dust shell, composed of amorphous carbon (amC) and silicon carbide (SiC). However, no clear SiC feature has been detected in the MATISSE visibilities. This might be caused by molecular absorption that can affect the shape of the SiC band at 11.3 micron. The appearance of the molecular shells is in good agreement with predictions from dynamical atmosphere models. For the first time, we co-located dust and molecules in the environment of an AGB star. We confirm that the molecules are located closer to the star than the dust. The MIRA images unveil the presence of a clumpy environment in the fuzzy emission region beyond 4.0 Rstar. Furthermore, with the available dynamic range and angular resolution, we did not detect the presence of a binary companion. Additional observations combining MATISSE and SAM-VISIR instrument should enable this detection in future studies. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Could not find figure 3D_spectra_geometrical_model_LM.png</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.10503-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.10503) | **Euclid: Calibrating photometric redshifts with spectroscopic  cross-correlations**  |
|| K. Naidoo, et al. -- incl., <mark>E. Franceschi</mark>, <mark>K. Jahnke</mark> |
|*Appeared on*| *2022-08-24*|
|*Comments*| *14 pages, 8 figures, submitted to A&A*|
|**Abstract**| Cosmological constraints from key probes of the Euclid imaging survey rely critically on the accurate determination of the true redshift distributions $n(z)$ of tomographic redshift bins. We determine whether the mean redshift $<z>$ of ten Euclid tomographic redshift bins can be calibrated to the Euclid target uncertainties of $\sigma(<z>)<0.002\,(1+z)$ via cross-correlation, with spectroscopic samples akin to those from the Baryon Oscillation Spectroscopic Survey (BOSS), Dark Energy Spectroscopic Instrument (DESI), and Euclid's NISP spectroscopic survey. We construct mock Euclid and spectroscopic galaxy samples from the Flagship simulation and measure small-scale clustering redshifts up to redshift $z<1.8$ with an algorithm that performs well on current galaxy survey data. The clustering measurements are then fitted to two $n(z)$ models: one is the true $n(z)$ with a free mean; the other a Gaussian Process modified to be restricted to non-negative values. We show that $<z>$ is measured in each tomographic redshift bin to an accuracy of order 0.01 or better. By measuring the clustering redshifts on subsets of the full Flagship area, we construct scaling relations that allow us to extrapolate the method performance to larger sky areas than are currently available in the mock. For the full expected Euclid, BOSS, and DESI overlap region of approximately 6000 deg$^{2}$, the uncertainties attainable by clustering redshifts exceeds the Euclid requirement by at least a factor of three for both $n(z)$ models considered, although systematic biases limit the accuracy. Clustering redshifts are an extremely effective method for redshift calibration for Euclid if the sources of systematic biases can be determined and removed, or calibrated-out with sufficiently realistic simulations. We outline possible future work, in particular an extension to higher redshifts with quasar reference samples. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Could not find figure plots/suppressed_GP_example.pdf</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.10997-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.10997) | **Reconstructing the lost eclipse events of the Saros spiral applying the  Draconic gearing on the Antikythera Mechanism. The impact of the gearing  errors on the pointers position**  |
|| Aristeidis Voulgaris, Christophoros Mouratidis, <mark>Andreas Vossinakis</mark> |
|*Appeared on*| *2022-08-24*|
|*Comments*| *Manuscript (26 pages and 13 figures) and Supplementary material (36 pages and 92 figures)*|
|**Abstract**| We present new observations concerning the procedure for the reconstruction of the lost eclipse events engraved in the Saros spiral cells of the Antikythera Mechanism. For the reconstructed eclipse events we applied the assumed, albeit missing, Draconic gearing of the Antikythera Mechanism, which was probably a part of the Mechanism gearing, representing the fourth lunar motion, the Draconic cycle. The Draconic gearing is very critical for the eclipse prediction and defines whether an eclipse will occur. For our research we created a program which presents the phase of the four lunar cycles and the position of the Draconic pointer relative to the ecliptic limits. After calibrating the software according to the preserved eclipse events, the lost eclipse events of the Saros spiral were calculated and discussed. The procedure for the calculation of the events times by using solely the Mechanism is also presented. The eccentricity error of a gear which is preserved on the ancient prototype is discussed. An experimental setup facilitated the analysis of the mechanical characteristics of gears with triangular teeth and the errors. The experimental study of the gears errors revealed the strong impact the Antikythera Mechanism pointers have on the results. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2208.10528.md
    + _build/html/tmp_2208.10528/./cooling_rates_grackle_vs_SG.png
exported in  _build/html/2208.10726.md
    + _build/html/tmp_2208.10726/./fig-01.png
    + _build/html/tmp_2208.10726/./fig-10.png
    + _build/html/tmp_2208.10726/./fig-12a.png
    + _build/html/tmp_2208.10726/./fig-12b.png
exported in  _build/html/2208.10854.md
    + _build/html/tmp_2208.10854/./Figures/tess_individual.png
    + _build/html/tmp_2208.10854/./Figures/RV.png
    + _build/html/tmp_2208.10854/./Figures/TOI4562_AO_Gemini.png
    + _build/html/tmp_2208.10854/./Figures/TOI_4562_SOAR_speckle.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\m}[1]{\underline{\underline{#1}}}$
$\newcommand{\wt}{\widetilde}$
$\newcommand{\cntext}[1]{\begin{CJK}{UTF8}{bsmi}#1\end{CJK}}$
$\newcommand{\ol}{\overline}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\vdag$}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{$\m$}[1]{\underline{\underline{#1}}}$
$\newcommand{$\wt$}{\widetilde}$
$\newcommand{$\cntext$}[1]{\begin{CJK}{UTF8}{bsmi}#1\end{CJK}}$
$\newcommand{$\ol$}{\overline}$</div>



<div id="title">

# 

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2208.10528-b31b1b.svg)](https://arxiv.org/abs/2208.10528)<mark>Appeared on: 2022-08-24</mark> - _submitted; comments welcome_

</div>
<div id="authors">

Chia-Yu Hu ($\cntext${胡家瑜}), et al. -- incl., <mark>Matthew C. Smith</mark>

</div>
<div id="abstract">

**Abstract:** We present a suite of high-resolution simulations of an isolated dwarf galaxy usingfour different hydrodynamical codes:{\sc Gizmo},{\sc Arepo},{\sc Gadget}, and{\sc Ramses}.All codes adopt the same physical modelwhichincludes radiative cooling,photoelectric heating,star formation,and supernova (SN) feedback.Individual SN explosions are directly resolved without resorting to sub-grid models,eliminating one of the major uncertainties in cosmological simulations.We findreasonable agreement onthe time-averaged star formation ratesas well as the joint density-temperature distributionsbetween all codes.However,the Lagrangian codes show significantly burstier star formation,larger supernova-driven bubbles,and stronger galactic outflowscompared to the Eulerian code.This is caused by the behavior in the dense, collapsing gas cloudswhen the Jeans length becomes unresolved:gas in Lagrangian codescollapses to much higher densitiesthan in Eulerian codes,as the latter is stabilized by the minimal cell size.Therefore,more of the gas cloud is converted to stars andSNe are much more clustered in the Lagrangian models,amplifying their dynamical impact.The differences between Lagrangian and Eulerian codescan be reducedby adopting a higher star formation efficiency in Eulerian codes,which significantly enhances SN clustering in the latter.Adopting a zero SN delay timereduces burstiness in all codes, resulting in vanishing outflowsas SN clustering is suppressed.

</div>

<div id="div_fig1">

<img src="tmp_2208.10528/./cooling_rates_grackle_vs_SG.png" alt="Fig1" width="100%"/>

**Figure 1. -** \textit{Upper left}:
		equilibrium temperature as a function of density for the {\sc Grackle} equilibrium module (dotted green),
		{\sc SGchem}(see text) with cosmic rate ionization rate $\zeta = 10^{-16} {\rm s^{-1}}$(solid blue),
		and {\sc SGchem} with $\zeta = 0$(dashed orange).
		\textit{Upper right}:
		photoelectric efficiency ($\epsilon_{\rm PE}$, blue) and electron abundance ($x_{\rm e}$, orange) as a function of density
		for {\sc SGchem} with $\zeta = 10^{-16} {\rm s^{-1}}$(solid) and $\zeta = 0$(dashed).
		\textit{Lower left}:
		individual cooling and heating processes as a function of density in {\sc SGchem} with $\zeta = 10^{-16} {\rm s^{-1}}$.
		\textit{Lower right}:
		same as lower left, but with $\zeta = 0$.
		Adopting a constant $\epsilon_{\rm PE}$ is a fair approximation
		of the more realistic situation where both photoelectric effect and cosmic ray ionization are present.
	 (*fig:cooling_gr_vs_SG*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\ic}{\color{red}}$
$\newcommand{\cf}{\color{blue}}$
$\newcommand{\bvc}[1]{\textbf{\color{green}#1}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\ic$}{\color{red}}$
$\newcommand{$\cf$}{\color{blue}}$
$\newcommand{$\bvc$}[1]{\textbf{\color{green}#1}}$
$\newcommand{$\thebibliography$}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# Truncated accretion discs in black hole X-ray binaries: dynamics and variability signatures

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2208.10726-b31b1b.svg)](https://arxiv.org/abs/2208.10726)<mark>Appeared on: 2022-08-24</mark> - _19 pages, 25 figures_

</div>
<div id="authors">

Indu K. Dihingia, Bhargav Vaidya, <mark><mark>Christian Fendt</mark></mark>

</div>
<div id="abstract">

**Abstract:** Variable features in black hole X-ray Binaries (BH-XRBs) are observed in different energy ranges and time scales. The physical origin of different spectral states in BH-XRBs and their relations with the underlying accretion disc are still elusive. To investigate the intermediate state of BH-XRBs during outburst, we simulate a truncated accretion disc around a Kerr black hole using a general relativistic magneto-hydrodynamical (GRMHD) framework under axisymmetry with adaptively refined mesh. Additionally, we have also carried out radiative transfer calculations for understanding the implications of disc dynamics on emission. Dynamically, the inner edge of the truncated accretion disc oscillates in a quasi-periodic fashion (QPO). The QPO frequency of oscillations$(\nu_{\rm QPO, max})$increases as the magnetic field strength and magnetic resistivity increase.  However, as the truncation radius increases,$\nu_{\rm QPO, max}$decreases. In our simulation models, frequency varies between$7\times(10M_{\odot}/M_{\rm BH})$Hz$\lesssim\nu_{\rm QPO, max}\lesssim20 \times (10M_{\odot}/M_{\rm BH})$Hz, which is in the range of low-frequency QPOs.We further find evidence of transient shocks in the highly accreting stage during oscillation. Such a transient shock acts as an extended hot post-shock corona around the black hole that has an impact on its radiative properties. The radiative transfer calculations show signatures of these oscillations in the form of modulation in the edge-brightened structure of the accretion disc.

</div>

<div id="div_fig1">

<img src="tmp_2208.10726/./fig-01.png" alt="Fig12" width="100%"/>

**Figure 12. -** Distribution of logarithmic normalised density distribution $(\rho/\rho^{\rm tr})$ and magnetic field lines for reference model 2D40AH in ideal MHD at times (a) $t=3240$, (b) $t=3800$, (c) $t=4080$, (d) $t=4220$, and (e) $t=4380$. (*fig-rho*)

</div>
<div id="div_fig2">

<img src="tmp_2208.10726/./fig-10.png" alt="Fig5" width="100%"/>

**Figure 5. -** Profile of (a) radial four velocity $(u^r)$, (b) logarithmic density $(\log(\rho/\rho^{\rm tr}))$, (c) gas pressure $(p/p^{\rm tr})$, (d) Bernoulli parameter $(-hu_t)$, and (e) Mach number ($M_s$) as a function of radial distance for model 2D40AH at time $t=4220$. The gray vertical line indicates the location of the shock transition. We show the shock transition region as an inset in panel (a). The gray horizontal dotted line in panel (e) corresponds to $M_s=1$.
The plots are shown along a parabolic streamline $y=c x^{2.5}$. See the text for more detail. (*fig-shock*)

</div>
<div id="div_fig3">

<img src="tmp_2208.10726/./fig-12a.png" alt="Fig17.1" width="50%"/><img src="tmp_2208.10726/./fig-12b.png" alt="Fig17.2" width="50%"/>

**Figure 17. -** Profiles of accretion rate $(\dot{M}_{\rm acc})$, mass flux through funnel $(\dot{M}_{\rm jet})$, mass flux through wind  $(\dot{M}_{\rm wind})$, and magnetic flux accumulated at the event horizon  $(\dot{\phi}_{\rm BH})$ for models with different $\beta_{\rm tr}$ and $r_{\rm tr}$. In panels a-d (left), plots are shown for models 2D40A, 2D40B, and 2D40C, corresponding to $\beta_{\rm tr}=100, 500,$ and $1000$, respectively, with $r_{\rm tr}=40$. In panels e-h (right), plots are shown for models 2D40A, 2D50, and 2D60, corresponding to $r_{\rm tr}=40, 50,$ and $60$, respectively, with $\beta_{\rm tr}=100$. The gray horizontal line in panels (d) and (h) correspond to $\dot{\phi}_{\rm BH}=15$. (*fig-fluxes*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\rev}$
$\newcommand{\TESS}{\emph{TESS}\xspace}$
$\newcommand{\toi}{TOI-4562\xspace}$
$\newcommand{\toib}{TOI-4562 b\xspace}$
$\newcommand{\tcvalue}{1456.874636^{+0.001172}_{-0.001249}}$
$\newcommand{\pervalue}{225.11757^{+0.00027}_{-0.00025}}$
$\newcommand{\rprstarvalue}{0.09450^{+0.00050}_{-0.00042}}$
$\newcommand{\incvalue}{89.72^{+0.17}_{-0.15}}$
$\newcommand{\secoswvalue}{0.37^{+0.11}_{-0.12}}$
$\newcommand{\sesinwvalue}{0.808^{+0.046}_{-0.052}}$
$\newcommand{\mpvalue}{0.0023 \pm 0.0005}$
$\newcommand{\mStarvalue}{1.218^{+0.054}_{-0.048}}$
$\newcommand{\teffvalue}{6270 \pm 50}$
$\newcommand{\loggvalue}{4.42 \pm 0.04}$
$\newcommand{\fehvalue}{0.24 \pm 0.10}$
$\newcommand{\agevalue}{300--400}$
$\newcommand{\parallaxvalue}{2.88356^{+0.0104}_{-0.0099}}$
$\newcommand{\meanvalue}{5352\pm13}$
$\newcommand{\jitavalue}{73^{+15}_{-15}}$
$\newcommand{\jitbvalue}{28.5^{+18.7}_{-20.9}}$
$\newcommand{\ebvvalue}{<0.01  (3\sigma)}$
$\newcommand{\rStarvalue}{1.126^{+0.014}_{-0.012}}$
$\newcommand{\densityStarvalue}{1.14^{+0.07}_{-0.037}}$
$\newcommand{\smarStarvalue}{147.1_{-6.1}^{+6.6}}$
$\newcommand{\smaAUvalue}{0.771^{+0.013}_{-0.013}}$
$\newcommand{\distvalue}{346.9\pm3.8}$
$\newcommand{\kampvalue}{353^{+73}_{-88}}$
$\newcommand{\mpjupvalue}{3.29^{+1.88}_{-0.82}}$
$\newcommand{\mpearthvalue}{768^{+170}_{-165}}$
$\newcommand{\rpjupvalue}{1.072_{-0.043}^{+0.044}}$
$\newcommand{\rpearthvalue}{11.72 \pm 0.28}$
$\newcommand{\densityvalue}{2.63^{+0.58}_{-0.56}}$
$\newcommand{\eccvalue}{0.81^{+0.05}_{-0.05}}$
$\newcommand{\wvalue}{63 \pm 5}$
$\newcommand{\bvalue}{0.29^{+0.24}_{-0.32}}$
$\newcommand{\tdepthvalue}{9066^{+225}_{-243}}$
$\newcommand{\tdurvalue}{4.23\pm{0.14}}$
$\newcommand{\teqvalue}{349\pm10}$
$\newcommand{\tperivalue}{731_{-34}^{+55}}$
$\newcommand{\tapovalue}{261_{-3}^{+2}}$
$\newcommand{\sepvalue}{35}$
$\newcommand{\teffCHIRON}{6096 \pm 50}$
$\newcommand{\loggCHIRON}{4.4 \pm 0.1}$
$\newcommand{\fehCHIRON}{0.1 \pm 0.1}$
$\newcommand{\vsiniCHIRON}{15.67 \pm 0.5}$
$\newcommand{\Protvalue}{3.82 \pm 0.05}$
$\newcommand{\incstarvalue}{89.7_{-12.7}^{+0.3}}$
$\newcommand$
$\newcommand{\teff}{\ensuremath{T_{\rm eff}}}$
$\newcommand{\met}{[M/H]}$
$\newcommand{\vsini}{v~sin i}$
$\newcommand{\kms}{ km s^{-1}}$
$\newcommand{\ms}{ m s^{-1}}$
$\newcommand{\radd}{ rad d^{-1}}$
$\newcommand{\degree}{^{\circ}}$
$\newcommand{\masyr}{mas yr^{-1}}$
$\newcommand{\Bmag}{B}$
$\newcommand{\Vmag}{V}$
$\newcommand{\Tmag}{T}$
$\newcommand{\gaiaG}{G}$
$\newcommand{\gaiaBP}{B_{\mathrm{p}}}$
$\newcommand{\gaiaRp}{R_{\mathrm{p}}}$
$\newcommand{\Jmag}{J}$
$\newcommand{\Hmag}{H}$
$\newcommand{\Kmag}{K_{\mathrm{s}}}$
$\newcommand{\Wi}{W_{\mathrm{1}}}$
$\newcommand{\Wii}{W_{\mathrm{2}}}$
$\newcommand{\Wiii}{W_{\mathrm{3}}}$
$\newcommand{\Rstar}{R_{\mathrm{\star}}}$
$\newcommand{\Mstar}{M_{\mathrm{\star}}}$
$\newcommand{\Lstar}{L_{\mathrm{\star}}}$
$\newcommand{\rhostar}{\rho_{\mathrm{\star}}}$
$\newcommand{\rhoSun}{\rho_{\mathrm{\odot}}}$
$\newcommand{\Msun}{M_{\odot}}$
$\newcommand{\Rsun}{R_{\odot}}$
$\newcommand{\TBJD}{BJD_{TDB}}$
$\newcommand{\REarth}{R_{\oplus}}$
$\newcommand{\MEarth}{M_{\oplus}}$
$\newcommand{\RJup}{R_{\mathrm{J}}}$
$\newcommand{\MJup}{M_{\mathrm{J}}}$
$\newcommand{\Porb}{P_{\mathrm{orb}}}$
$\newcommand{\Prot}{P_{\star}}$
$\newcommand{\Mp}{M_{\mathrm{p}}}$
$\newcommand{\Rp}{R_{\mathrm{p}}}$
$\newcommand{\ars}{a/R_{\mathrm{\star}}}$
$\newcommand{\tc}{T_{\mathrm{c}}}$
$\newcommand{\secosw}{\sqrt{e}\cos\omega}$
$\newcommand{\sesinw}{\sqrt{e}\sin\omega}$
$\newcommand{\offsetchiron}{\gamma_{\mathrm{rel}}}$
$\newcommand{\jitterchiron}{\sigma_{\mathrm{j}}}$
$\newcommand{\Kamp}{K_{\mathrm{amp}}}$
$\newcommand{\tikzcircle}[2][red,fill=red]{\tikz[baseline=-0.5ex]\draw[#1,radius=#2] (0,0) circle ;}$
$\newcommand{\USQ}{University of Southern Queensland, Centre for Astrophysics, West Street, Toowoomba, QLD 4350 Australia}$
$\newcommand{\PennStateAA}{Department of Astronomy \& Astrophysics, 525 Davey Laboratory, The Pennsylvania State University, University Park, PA, 16802, USA}$
$\newcommand{\PennStateCEHW}{Center for Exoplanets and Habitable Worlds, 525 Davey Laboratory, The Pennsylvania State University, University Park, PA, 16802, USA}$
$\newcommand{\CCA}{Center for Computational Astrophysics, Flatiron Institute, 162 Fifth Avenue, New York, NY 10010, USA}$
$\newcommand{\CfA}{Center for Astrophysics \textbar  Harvard \& Smithsonian, 60 Garden Street, Cambridge, MA 02138, USA}$
$\newcommand{\Caltech}{Cahill Center for Astrophysics, California Institute of Technology, Pasadena, CA 91125, USA}$
$\newcommand{\IRAP}{Institut de Recherche en Astrophysique et Planétologie, Université de Toulouse, CNRS, CNES, 14 avenue Edouard Belin, 31400 Toulouse, France}$
$\newcommand{\LiegeUni}{Astrobiology Research Unit, Université de Liège, 19C Allée du 6 Août, 4000 Liège, Belgium}$
$\newcommand{\MITAtmo}{Department of Earth, Atmospheric and Planetary Science, Massachusetts Institute of Technology, 77 Massachusetts Avenue, Cambridge, MA 02139, USA}$
$\newcommand{\IACSpain}{Instituto de Astrof\'isica de Canarias (IAC), Calle V\'ia L\'actea s/n, 38200, La Laguna, Tenerife, Spain}$
$\newcommand{\QMU}{Astronomy Unit, Queen Mary University of London, Mile End Road, London E1 4NS, UK}$
$\newcommand{\CAM}{Astrophysics Group, Cavendish Laboratory, J.J. Thomson Avenue, Cambridge CB3 0HE, UK}$
$\newcommand{\WarwickUni}{Department of Physics, University of Warwick, Gibbet Hill Road, Coventry CV4 7AL, UK}$
$\newcommand{\CEHWarwick}{Centre for Exoplanets and Habitability, University of Warwick, Gibbet Hill Road, Coventry CV4 7AL, UK}$
$\newcommand{\BirminghamUni}{School of Physics \& Astronomy, University of Birmingham, Edgbaston, Birmingham, B15 2TT, UK}$
$\newcommand{\AIUniChile}{Facultad de Ingeniería y Ciencias, Universidad Adolfo Ib\'a\~nez, Av. Diagonal las Torres 2640, Pe\~nalol\'en, Santiago, Chile}$
$\newcommand{\MIAChile}{Millennium Institute of Astrophysics MAS, Nuncio Monsenor Sotero Sanz 100, Of. 104, Providencia, Santiago, Chile}$
$\newcommand{\MaxPlank}{Max Planck Institute for Astronomy, K\"onigstuhl 17, D-69117—Heidelberg, Germany}$
$\newcommand{\KeeleUni}{Astrophysics Group, Keele University, Staffordshire, ST5 5BG, UK}$
$\newcommand{\kavlimit}{Department of Physics and Kavli Institute for Astrophysics and Space Research, Massachusetts Institute of Technology, Cambridge, MA 02139, USA}$
$\newcommand{\NESI}{NASA Exoplanet Science Institute, Caltech/IPAC, 1200 E. California Ave, Pasadena, CA 91125, USA}$
$\newcommand{\WisconsinMadisonUni}{Department of Astronomy, University of Wisconsin-Madison, Madison, WI 53706, USA}$
$\newcommand{\UTAustin}{Department of Astronomy, The University of Texas at Austin, Austin, TX 78712, USA}$
$\newcommand{\plahunters}{Citizen Scientist, Zooniverse c/o University of Oxford, Keble Road, Oxford OX1 3RH, UK}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\vdag$}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{$\rev$}$
$\newcommand{$\TESS$}{\emph{TESS}$\xspace$}$
$\newcommand{$\toi$}{TOI-4562$\xspace$}$
$\newcommand{$\toi$b}{TOI-4562 b$\xspace$}$
$\newcommand{$\tcvalue$}{1456.874636^{+0.001172}_{-0.001249}}$
$\newcommand{$\pervalue$}{225.11757^{+0.00027}_{-0.00025}}$
$\newcommand{$\rprstarvalue$}{0.09450^{+0.00050}_{-0.00042}}$
$\newcommand{$\incvalue$}{89.72^{+0.17}_{-0.15}}$
$\newcommand{$\secoswvalue$}{0.37^{+0.11}_{-0.12}}$
$\newcommand{$\sesinwvalue$}{0.808^{+0.046}_{-0.052}}$
$\newcommand{$\mpvalue$}{0.0023 \pm 0.0005}$
$\newcommand{$\mStarvalue$}{1.218^{+0.054}_{-0.048}}$
$\newcommand{$\teffvalue$}{6270 \pm 50}$
$\newcommand{$\loggvalue$}{4.42 \pm 0.04}$
$\newcommand{$\fehvalue$}{0.24 \pm 0.10}$
$\newcommand{$\agevalue$}{300--400}$
$\newcommand{$\parallaxvalue$}{2.88356^{+0.0104}_{-0.0099}}$
$\newcommand{$\meanvalue$}{5352\pm13}$
$\newcommand{$\jitavalue$}{73^{+15}_{-15}}$
$\newcommand{$\jitbvalue$}{28.5^{+18.7}_{-20.9}}$
$\newcommand{$\ebvvalue$}{<0.01  (3\sigma)}$
$\newcommand{$\rStarvalue$}{1.126^{+0.014}_{-0.012}}$
$\newcommand{$\densityStarvalue$}{1.14^{+0.07}_{-0.037}}$
$\newcommand{$\smarStarvalue$}{147.1_{-6.1}^{+6.6}}$
$\newcommand{$\smaAUvalue$}{0.771^{+0.013}_{-0.013}}$
$\newcommand{$\distvalue$}{346.9\pm3.8}$
$\newcommand{$\kampvalue$}{353^{+73}_{-88}}$
$\newcommand{$\mpjupvalue$}{3.29^{+1.88}_{-0.82}}$
$\newcommand{$\mpearthvalue$}{768^{+170}_{-165}}$
$\newcommand{$\rpjupvalue$}{1.072_{-0.043}^{+0.044}}$
$\newcommand{$\rpearthvalue$}{11.72 \pm 0.28}$
$\newcommand{$\densityvalue$}{2.63^{+0.58}_{-0.56}}$
$\newcommand{$\eccvalue$}{0.81^{+0.05}_{-0.05}}$
$\newcommand{$\wvalue$}{63 \pm 5}$
$\newcommand{$\bvalue$}{0.29^{+0.24}_{-0.32}}$
$\newcommand{$\tdepthvalue$}{9066^{+225}_{-243}}$
$\newcommand{$\tdurvalue$}{4.23\pm{0.14}}$
$\newcommand{$\teqvalue$}{349\pm10}$
$\newcommand{$\tperivalue$}{731_{-34}^{+55}}$
$\newcommand{$\tapovalue$}{261_{-3}^{+2}}$
$\newcommand{$\sepvalue$}{35}$
$\newcommand{$\teffCHIRON$}{6096 \pm 50}$
$\newcommand{$\loggCHIRON$}{4.4 \pm 0.1}$
$\newcommand{$\fehCHIRON$}{0.1 \pm 0.1}$
$\newcommand{$\vsiniCHIRON$}{15.67 \pm 0.5}$
$\newcommand{$\Protvalue$}{3.82 \pm 0.05}$
$\newcommand{$\incstarvalue$}{89.7_{-12.7}^{+0.3}}$
$\newcommand$
$\newcommand{$\teff$}{$\ensuremath${T_{\rm eff}}}$
$\newcommand{$\met$}{[M/H]}$
$\newcommand{$\vsini$}{v~sin i}$
$\newcommand{$\kms$}{ km s^{-1}}$
$\newcommand{$\ms$}{ m s^{-1}}$
$\newcommand{$\radd$}{ rad d^{-1}}$
$\newcommand{$\degree$}{^{\circ}}$
$\newcommand{$\masyr$}{mas yr^{-1}}$
$\newcommand{$\Bmag$}{B}$
$\newcommand{$\Vmag$}{V}$
$\newcommand{$\Tmag$}{T}$
$\newcommand{$\gaiaG$}{G}$
$\newcommand{$\gaiaBP$}{B_{\mathrm{p}}}$
$\newcommand{$\gaiaRp$}{R_{\mathrm{p}}}$
$\newcommand{$\Jmag$}{J}$
$\newcommand{$\Hmag$}{H}$
$\newcommand{$\Kmag$}{K_{\mathrm{s}}}$
$\newcommand{$\Wi$}{W_{\mathrm{1}}}$
$\newcommand{$\Wi$i}{W_{\mathrm{2}}}$
$\newcommand{$\Wi$ii}{W_{\mathrm{3}}}$
$\newcommand{$\Rstar$}{R_{\mathrm{\star}}}$
$\newcommand{$\Mstar$}{M_{\mathrm{\star}}}$
$\newcommand{$\Lstar$}{L_{\mathrm{\star}}}$
$\newcommand{$\rhostar$}{\rho_{\mathrm{\star}}}$
$\newcommand{$\rhoSun$}{\rho_{\mathrm{\odot}}}$
$\newcommand{$\Msun$}{M_{\odot}}$
$\newcommand{$\Rsun$}{R_{\odot}}$
$\newcommand{$\TBJD$}{BJD_{TDB}}$
$\newcommand{$\REarth$}{R_{\oplus}}$
$\newcommand{$\MEarth$}{M_{\oplus}}$
$\newcommand{$\RJup$}{R_{\mathrm{J}}}$
$\newcommand{$\MJup$}{M_{\mathrm{J}}}$
$\newcommand{$\Porb$}{P_{\mathrm{orb}}}$
$\newcommand{$\Prot$}{P_{\star}}$
$\newcommand{$\Mp$}{M_{\mathrm{p}}}$
$\newcommand{$\Rp$}{R_{\mathrm{p}}}$
$\newcommand{$\ars$}{a/R_{\mathrm{\star}}}$
$\newcommand{$\tc$}{T_{\mathrm{c}}}$
$\newcommand{$\secosw$}{\sqrt{e}\cos\omega}$
$\newcommand{$\sesinw$}{\sqrt{e}\sin\omega}$
$\newcommand{$\offsetchiron$}{\gamma_{\mathrm{rel}}}$
$\newcommand{$\jitterchiron$}{\sigma_{\mathrm{j}}}$
$\newcommand{$\Kamp$}{K_{\mathrm{amp}}}$
$\newcommand{$\tikzcircle$}[2][red,fill=red]{\tikz[baseline=-0.5ex]\draw[#1,radius=#2] (0,0) circle ;}$
$\newcommand{$\USQ$}{University of Southern Queensland, Centre for Astrophysics, West Street, Toowoomba, QLD 4350 Australia}$
$\newcommand{$\PennStateAA$}{Department of Astronomy \& Astrophysics, 525 Davey Laboratory, The Pennsylvania State University, University Park, PA, 16802, USA}$
$\newcommand{$\PennStateCEHW$}{Center for Exoplanets and Habitable Worlds, 525 Davey Laboratory, The Pennsylvania State University, University Park, PA, 16802, USA}$
$\newcommand{$\CCA$}{Center for Computational Astrophysics, Flatiron Institute, 162 Fifth Avenue, New York, NY 10010, USA}$
$\newcommand{$\CfA$}{Center for Astrophysics \textbar  Harvard \& Smithsonian, 60 Garden Street, Cambridge, MA 02138, USA}$
$\newcommand{$\Caltech$}{Cahill Center for Astrophysics, California Institute of Technology, Pasadena, CA 91125, USA}$
$\newcommand{$\IRAP$}{Institut de Recherche en Astrophysique et Planétologie, Université de Toulouse, CNRS, CNES, 14 avenue Edouard Belin, 31400 Toulouse, France}$
$\newcommand{$\LiegeUni$}{Astrobiology Research Unit, Université de Liège, 19C Allée du 6 Août, 4000 Liège, Belgium}$
$\newcommand{$\MITAtmo$}{Department of Earth, Atmospheric and Planetary Science, Massachusetts Institute of Technology, 77 Massachusetts Avenue, Cambridge, MA 02139, USA}$
$\newcommand{$\IACSpain$}{Instituto de Astrof\'isica de Canarias (IAC), Calle V\'ia L\'actea s/n, 38200, La Laguna, Tenerife, Spain}$
$\newcommand{$\QMU$}{Astronomy Unit, Queen Mary University of London, Mile End Road, London E1 4NS, UK}$
$\newcommand{$\CAM$}{Astrophysics Group, Cavendish Laboratory, J.J. Thomson Avenue, Cambridge CB3 0HE, UK}$
$\newcommand{$\WarwickUni$}{Department of Physics, University of Warwick, Gibbet Hill Road, Coventry CV4 7AL, UK}$
$\newcommand{$\CEHWarwick$}{Centre for Exoplanets and Habitability, University of Warwick, Gibbet Hill Road, Coventry CV4 7AL, UK}$
$\newcommand{$\BirminghamUni$}{School of Physics \& Astronomy, University of Birmingham, Edgbaston, Birmingham, B15 2TT, UK}$
$\newcommand{$\AIUniChile$}{Facultad de Ingeniería y Ciencias, Universidad Adolfo Ib\'a\~nez, Av. Diagonal las Torres 2640, Pe\~nalol\'en, Santiago, Chile}$
$\newcommand{$\MIAChile$}{Millennium Institute of Astrophysics MAS, Nuncio Monsenor Sotero Sanz 100, Of. 104, Providencia, Santiago, Chile}$
$\newcommand{$\MaxPlank$}{Max Planck Institute for Astronomy, K\"onigstuhl 17, D-69117—Heidelberg, Germany}$
$\newcommand{$\KeeleUni$}{Astrophysics Group, Keele University, Staffordshire, ST5 5BG, UK}$
$\newcommand{$\kavlimit$}{Department of Physics and Kavli Institute for Astrophysics and Space Research, Massachusetts Institute of Technology, Cambridge, MA 02139, USA}$
$\newcommand{$\NESI$}{NASA Exoplanet Science Institute, Caltech/IPAC, 1200 E. California Ave, Pasadena, CA 91125, USA}$
$\newcommand{$\Wi$sconsinMadisonUni}{Department of Astronomy, University of Wisconsin-Madison, Madison, WI 53706, USA}$
$\newcommand{$\UTAustin$}{Department of Astronomy, The University of Texas at Austin, Austin, TX 78712, USA}$
$\newcommand{$\plahunters$}{Citizen Scientist, Zooniverse c/o University of Oxford, Keble Road, Oxford OX1 3RH, UK}$</div>



<div id="title">

# : A highly eccentric temperate Jupiter analog orbiting a young field star.

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2208.10854-b31b1b.svg)](https://arxiv.org/abs/2208.10854)<mark>Appeared on: 2022-08-24</mark> - _23 pages, 12 figures, 4 tables. Submitted to the Astrophysical Journal (23/08/2022)_

</div>
<div id="authors">

<mark><mark>Alexis Heitzmann</mark></mark>, et al. -- incl., <mark><mark>Melissa Hobson</mark></mark>, <mark><mark>Coel Hellier</mark></mark>, <mark><mark>Carl Ziegler</mark></mark>

</div>
<div id="abstract">

**Abstract:** We report the discovery of$\toi$b(TIC-349576261), a Jovian planet orbiting a young F7V-type star, younger than the Praesepe/Hyades clusters ($\sim$300--400 Myr). This planet stands out because of its unusually long orbital period for transiting planets with known masses ($\Porb$=$\pervalue$days), and because it has a substantial eccentricity ($e$=$\eccvalue$). The location of$\toi$near the southern continuous viewing zone of$\TESS$allowed observations throughout 25 sectors, enabling an unambiguous period measurement from$\TESS$alone. Alongside the four available$\TESS$transits, we performed follow-up photometry using the South African Astronomical Observatory node of the Las Cumbres Observatory, and spectroscopy with the CHIRON spectrograph on the 1.5 m SMARTS telescope. We measure a radius of$\rpjupvalue$\RJup, and a mass of$\mpjupvalue$\MJupfor$\toi$b. The radius of the planet is consistent with contraction models describing the early evolution of the size of giant planets. We detect tentative transit timing variations at the$\sim$20 min level from five transit events, favouring the presence of a companion that could explain the dynamical history of this system if confirmed by future follow-up observations. With its current orbital configuration, tidal timescales are too long for$\toi$bto become a hot-Jupiter via high eccentricity migration, though it is not excluded that interactions with the possible companion could modify$\toi$b's eccentricity and trigger circularization. The characterisation of more such young systems is essential to set constraints on models describing giant planet evolution.

</div>

<div id="div_fig1">

<img src="tmp_2208.10854/./Figures/tess_individual.png" alt="Fig9" width="100%"/>

**Figure 9. -** \textbf{Top} Individual transits from $\TESS$ from Sectors 5 and 13 at 30 minute cadence, and from Sectors 30 and 38 at 2 minute cadence. The red circles indicate the measured data for sectors 5 and 13, and the binned data at 30 minute cadence for Sectors 30 and 38. The 2-minute cadence data for Sectors 30 and 38 are plotted as brown points. The best fit model, incorporating the transit timing variations in Section \ref{sec:analysis}, and out-of-transit trends, are shown by the red lines. \textbf{Bottom left} The phase folded $\TESS$ transit and best fit model. \textbf{Bottom right} The combined follow-up LCOGT 1 m observations from 2022-01-03 in $i'$ band. (*fig:transits*)

</div>
<div id="div_fig2">

<img src="tmp_2208.10854/./Figures/RV.png" alt="Fig10" width="100%"/>

**Figure 10. -** \textbf{Top}: 2-year radial velocity time series of $\toi$ obtained with the CHIRON spectrograph (brown points) with the associated error bars. The Keplerian orbit fit from our global modelling is shown in red. Transits are highlighted in light brown. \textbf{Bottom}: Phase folded RVs (brown) with the Keplerian orbit best fits in red. (*fig:RV*)

</div>
<div id="div_fig3">

<img src="tmp_2208.10854/./Figures/TOI4562_AO_Gemini.png" alt="Fig1.1" width="50%"/><img src="tmp_2208.10854/./Figures/TOI_4562_SOAR_speckle.png" alt="Fig1.2" width="50%"/>

**Figure 1. -** \textbf{Top} High resolution image of $\toi$ obtained with the Zorro camera attached to the 8.1 m Gemini South telescope. The blue and red curves show difference in magnitude as a function of orbital separation from $\toi$ obtained at wavelengths of respectively 562 and 832 nm. The inset plot shows the reconstructed image at 832 nm where no companion is detected. \textbf{Bottom} SOAR HRCam high resolution imaging of $\toi$. The difference in magnitude as a function of orbital separation from $\toi$ is shown by the black line and the autocorrelation function on the inset image. There is no sign of a stellar sized companion to $\toi$.  (*fig:highres*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

23  publications in the last 7 days.
	 _build/html/2208.10854.md
	 _build/html/2208.10726.md
	 _build/html/2208.10528.md
	 _build/html/2208.09335.md
	 _build/html/2208.08872.md
	 _build/html/2204.06393.md
	 _build/html/2204.03335.md
	 _build/html/2204.03253.md
	 _build/html/2204.02998.md
	 _build/html/2204.02109.md
	 _build/html/2204.02017.md
	 _build/html/2204.01824.md
	 _build/html/2204.01758.md
	 _build/html/2204.01245.md
	 _build/html/2204.00793.md
	 _build/html/2204.00342.md
	 _build/html/2203.16959.md
	 _build/html/2203.16856.md
	 _build/html/2203.16735.md
	 _build/html/2203.16734.md
	 _build/html/2203.16504.md
	 _build/html/2203.15822.md
	 _build/html/2203.15811.md


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers